# API Scavenger Game

## Challenge 1: Fork Languages

You will find out how many programming languages are used among all the forks created from the main lab repo of your bootcamp.

In [1]:
import requests
import json
import pandas as pd
from pandas.io.json import json_normalize
import collections
import base64


Assuming the main lab repo is ironhack-datalabs/madrid-oct-2018, you will:

#### 1. Obtain the full list of forks created from the main lab repo via Github API.

To list forks, we can use the GET method. As explained in the GitHub API documentation, we need to make the request to: GET /repos/:owner/:repo/forks.

In [2]:
response = requests.get('https://api.github.com/repos/ironhack-datalabs/dataV2-labs/forks', data={'If-Modified-Since': 'Mon, 01 Jan 2000 00:00:00 GMT'})
data = pd.DataFrame(response.json())

In [3]:
def flatten(data, col_list):
    for column in col_list:
        flattened = pd.DataFrame(dict(data[column])).transpose()
        columns = [str(col) for col in flattened.columns]
        flattened.columns = [column + '_' + colname for colname in columns]
        data = pd.concat([data, flattened], axis=1)
        data = data.drop(column, axis=1)
    return data

In [4]:
len(data['fork'])

30

#### 2. Loop the JSON response to find out the language attribute of each fork. Use an array to store the language attributes of each fork.
Hint: Each language should appear only once in your array.
rint the language array. It should be something like: ["Python", "Jupyter Notebook", "HTML"]

In [5]:
data['language'].value_counts()

Jupyter Notebook    18
Name: language, dtype: int64

## Challenge 2: Count Commits
Count how many commits were made in the month of october of 2018.
#### 1. Obtain all the commits made in October 2018 via API, which is a JSON array that contains multiple commit objects.

In [6]:
response = requests.get('https://api.github.com/repos/ironhack-datalabs/dataV2-labs/commits', data={'If-Modified-Since': 'Mon, 01 Jan 2018 00:00:00 GMT'} )
data = pd.DataFrame(response.json())

In [7]:
data['commit']

0     {'author': {'name': 'kelsey-ironhack', 'email'...
1     {'author': {'name': 'kelsey-ironhack', 'email'...
2     {'author': {'name': 'kelsey-ironhack', 'email'...
3     {'author': {'name': 'kbartonhenry', 'email': '...
4     {'author': {'name': 'kbartonhenry', 'email': '...
5     {'author': {'name': 'kbartonhenry', 'email': '...
6     {'author': {'name': 'kelsey-ironhack', 'email'...
7     {'author': {'name': 'kelsey-ironhack', 'email'...
8     {'author': {'name': 'kelsey-ironhack', 'email'...
9     {'author': {'name': 'kelsey-ironhack', 'email'...
10    {'author': {'name': 'kelsey-ironhack', 'email'...
11    {'author': {'name': 'kelsey-ironhack', 'email'...
12    {'author': {'name': 'kelsey-ironhack', 'email'...
13    {'author': {'name': 'kelsey-ironhack', 'email'...
14    {'author': {'name': 'kelsey-ironhack', 'email'...
15    {'author': {'name': 'kelsey-ironhack', 'email'...
16    {'author': {'name': 'kelsey-ironhack', 'email'...
17    {'author': {'name': 'kelsey-ironhack', 'em

#### 2. Count how many commit objects are contained in the array.

In [8]:
len(data['commit'])

30

## Challenge 3: Hidden Cold Joke

Using Python, call Github API to find out the cold joke contained in the 24 secret files in the following repo:

https://github.com/ironhack-datalabs/scavenger

The filenames of the secret files contain .scavengerhunt and they are scattered in different directories of this repo. The secret files are named from .0001.scavengerhunt to .0024.scavengerhunt. They are scattered randomly throughout this repo. You need to search for these files by calling the Github API, not searching the local files on your computer.

#### 1. Find the secret files.

In [9]:
response = requests.get('https://api.github.com/search/code?q=extension:scavengerhunt+repo:ironhack-datalabs/scavenger')
response.ok

True

In [10]:
df = pd.DataFrame(response.json())

#### 2.  Sort the filenames ascendingly.

In [11]:
scavlist = sorted(flatten(df,['items'])['items_name'])

In [18]:
scavlist

['.0001.scavengerhunt',
 '.0002.scavengerhunt',
 '.0003.scavengerhunt',
 '.0004.scavengerhunt',
 '.0005.scavengerhunt',
 '.0006.scavengerhunt',
 '.0007.scavengerhunt',
 '.0008.scavengerhunt',
 '.0009.scavengerhunt',
 '.0010.scavengerhunt',
 '.0011.scavengerhunt',
 '.0012.scavengerhunt',
 '.0013.scavengerhunt',
 '.0014.scavengerhunt',
 '.0015.scavengerhunt',
 '.0016.scavengerhunt',
 '.0017.scavengerhunt',
 '.0018.scavengerhunt',
 '.0019.scavengerhunt',
 '.0020.scavengerhunt',
 '.0021.scavengerhunt',
 '.0022.scavengerhunt',
 '.0023.scavengerhunt',
 '.0024.scavengerhunt']

#### 3. Read the content of each secret files into an array of strings.
Since the response is encoded, you will need to send the following information in the header of your request:
````python
headers = {'Accept': 'application/vnd.github.v3.raw'}
````

In [12]:
scavenger = scavlist[0]

In [13]:
response = requests.get('https://api.github.com/repos/ironhack-datalabs/scavenger/git/trees/master?recursive=1')
response.ok

True

In [14]:
data = pd.DataFrame(response.json())

In [15]:
flattened = flatten(data, ['tree'])

In [16]:
linklist = []
for t in flattened['tree_path']:
    if t.endswith('scavengerhunt'):
        linklist.append(t)

In [17]:
file_plus_content = []
for l in linklist:
    link = 'https://api.github.com/repos/ironhack-datalabs/scavenger/contents/'+l
    
    response = requests.get(link, auth=('user', 'password'))
    response.ok
    file_plus_content.append([l, response.json()['content']])

KeyError: 'content'

In [ ]:
new_list = []
for l in file_plus_content:
    new_list.append([l[0][6:], l[1]])

In [ ]:
new_list = sorted(new_list)

#### 4. Concatenate the strings in the array separating each two with a whitespace.

In [ ]:
# your code fulltext = []
for f in new_list:
    fulltext.append(base64.b64decode(f[1]))

#### 5. Print out the joke.

In [ ]:
joke = b' '.join(fulltext).decode("utf-8")

In [ ]:
''.join(joke.splitlines())